# Run general analysis on phagophore membranes

When first ran need to pip install:
pyvista
pyvistaqt
mrcfile

In [ ]:
#imports
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

import pyvista as pv
from pyvistaqt import BackgroundPlotter
import os
import mrcfile as tf
import math
from numpy.linalg import inv, eig
from math import sqrt
from scipy.spatial import cKDTree
from tqdm import tqdm
from mpl_toolkits.mplot3d import Axes3D
from functools import partial
from scipy.optimize import minimize


Functions

In [ ]:
#if vol is a 3d numpy array representing a labelled volume where labels are e.g. 
#ER: 1 and Golgi: 2 then points["ER"] will be a 2D array of all the coordinates in vol where the value is 1.
def load_segmentation_tif(fname, tag, return_vol=False, verbose=False):
    """Load Amira segmentation saved as tif, and get all segmented points."""
    
    with tf.open(fname, permissive=True) as mrc:
        vol = np.copy(mrc.data) # load data (3d tif, or use mrcfile for mrc)
    vol = np.transpose(vol, axes=[2,1,0]) # Sort axes to xyz
    if verbose:
        print("Input volume shape is {}".format(vol.shape))

    # Extract points with labels saved in the dictionary "labels"
    # points = {name: np.vstack(np.where(vol == label)).T for name, label in labels.items()}    
    points = {tag: np.vstack(np.where(vol != 0)).T}
    if return_vol:
        return points, vol
    
    return points


Sphere functions

In [ ]:
def sphere_fit_rmse(points, center, radius):
    """Calculate root mean square error of a sphere fit to points."""
    distances_to_center = np.sqrt( np.sum((points - center)**2, axis=1) )
    residuals = distances_to_center - radius
    residual_sum_squares = np.sum( np.power(residuals, 2) )
    rmse = np.sqrt(residual_sum_squares / len(residuals))
    return rmse

In [ ]:
def ls_sphere_3(points):
    """
    Least squares fit of sphere to points.
    
    Modified code following approach described in:
    https://jekel.me/2015/Least-Squares-Sphere-Fit/
    
    Approach:
    Starting point: (x-x0)^2 + (y-y0)^2 + (z-z0)^2 = r^2
    Rearranged to: x^2 + y^2 + z^2 = 2x*x0 + 2y*y0 + 2z*z0 + 1*(r^2 -x0^2 -y0^2 -z0^2)
    --> Matrix:           f        =  A c
    with  f: nx1 vector with (xi^2 + yi^2 + zi^2) in i=1:n rows
          A: nx4 matrix with rows (2xi, 2yi, 2zi, 1)
          c: 4x1 target matrix [x0, y0, z0, (r^2 - x0^2 - y0^2 - z0^2)]

    
    Parameters
    ----------
    points : numpy.ndarray
        (n,3) array of point xyz coordinates.

    Returns
    -------
    center : numpy.ndarray
        xyz coordinates of sphere center.
    radius : float
        Sphere radius.
    rmse : float
        Root mean square error of fit.

    """
    # Assemble the A matrix
    A = np.append(2*points, np.ones( (points.shape[0],1) ), axis=1 ) # (2xi, 2yi, 2zi, 1)
    # Assemble f
    f = np.sum( np.power(points,2), axis=1) # x^2 + y^2 + z^2
    
    # Solve linear equation system Ac = f using least-squares algorithm
    c, residuals, rank, singval = np.linalg.lstsq(A,f)
    # Extract the center (first three values in c)
    center = c[0:3]
    # solve for the radius, c[3] = r^2 - x0^2 - y0^2 - z0^2
        # --> radius = sqrt( c[3] + x0^2 + y0^2 + z0^2 )
    radius = math.sqrt( c[3] + np.sum( np.power(center,2) ) )
    
    # determine RMSE by comparing distances of points to center to the radius
    rmse = sphere_fit_rmse(points, center, radius)
    
    return center, radius, rmse

Ellipsoid fitting Functions

In [ ]:
def project_to_simplex(y):
    """
    Project an n-dim vector y to the simplex Dn.
    
    Dn = { x : x n-dim, 1 >= x >= 0, sum(x) = 1}
    (c) Xiaojing Ye, 2011
    
    Algorithm is explained as in the linked document http://arxiv.org/abs/1101.6081
    """
    bget = False; 
    s = -np.sort(-y) # Workaround to get sorting in descending order
    
    tmpsum = 0
    for i, si in enumerate(s[:-1]):
        tmpsum += si
        tmax = (tmpsum-1)/(i+1)
        if tmax >= s[i+1]:
            bget = True
            break
    
    if ~bget:
        tmax = (tmpsum + s[-1] - 1) / len(y)
    
    x = np.maximum(y-tmax, 0) # element-wise comparison, takes larger value for each element
    
    return x

In [ ]:
def project_on_B(q0):
    """Project q0 on B.
    calls: project_to_simplex"""
    Q0 = np.array([[q0[0],          q0[3]/sqrt(2),  q0[4]/sqrt(2)],
                   [q0[3]/sqrt(2),  q0[1],          q0[5]/sqrt(2)],
                   [q0[4]/sqrt(2),  q0[5]/sqrt(2),  q0[2]]])
    #q0 encodes the quadratic terms of an ellipsoid in a vectorized form:
    #QO is reshaped into symmetric 3 x 3 matrix
    [s0,U]=eig(Q0)
    #This step decomposes the quadratic form into its principal axes and corresponding scales.
    s = project_to_simplex(s0)
    #A simplex projection ensures that the eigenvalues are non-negative and sum to 1 (or some other normalization), enforcing positive semidefiniteness and a normalized scale.
    S = np.diag(s)
    Q = U @ S @ U.T
    #rebuild
    
    q = np.concatenate( (np.array([Q[0,0], Q[1,1], Q[2,2], sqrt(2)* Q[1,0], sqrt(2)* Q[2,0], sqrt(2)* Q[2,1]]), 
                         q0[6:] ) )
    #convert Q back to q
    
    return q

In [ ]:
def fit_ellipsoid_DR_SVD(x, n_iter=1000):
    """
    Fit an ellipsoid to points.
    
    Given a set of points x=(x1,..,xn), this function finds a fitting
    ellipsoid in 3D, by using the approach proposed in the companion paper. 
    This method is not affine invariant. DR stands for Douglas-Rachford
    (Lions-Mercier would be more accurate).
    
    The output ellipsoid E is described implicitely by a triplet (A,b,c):
    E={x in R^3, <Ax,x> + <b,x> + c=0}
    or alternatively by vector q =(a11,a22,a33,sqrt(2)a12,sqrt(2)a13,sqrt(2)a23,b1,b2,b3,c).
    
    INPUT:
    - x: set of coordinates of size 2xn.
    - n_iter: number of iterations in Douglas-Rachford algorithm.
    OUTPUT:
    - A,b,c : matrix, vector, scalar describing the ellipsoid.
    - q: (a11,a22,a33,sqrt(2)a12,sqrt(2)a13,sqrt(2)a23,b1,b2,b3,c). i.e. ellipsoid equation
    - CF: Cost function wrt to iterations.

    calls:
    Project_on_B
    """
    # Find SVD of x and change coordinates
    x_mean = x.mean(axis=0) #centroid of our object
    x_centered = x - x_mean # Center points around origin
    x_eval, x_evec = eig(x_centered.T @ x_centered) # Singular value decomposition. 
    #calculate eigen values i.e. x_eval how spreadout the data is. x_evec = eigen vector, i.e. principle axes of the points
    
    P = np.diagflat(np.power(x_eval, -0.5)) @ x_evec.T # Transformation matrix for normalization
    
    x_norm = P @ x_centered.T # normalize x

    #arranges points around the centroid, aligns the principle axes and makes the data isotropic - preprocessing. 
    #principal axes = 1. direction point cloid is most spread out. 2. direction of next more spreadout. 3. orthagonal to both. they explain variation in data
    
    # Assemble matrix D. i.e. equations to solve ellipsoid. 
    D = np.vstack( (x_norm**2, 
                    sqrt(2)*x_norm[0,:]* x_norm[1,:],
                    sqrt(2)*x_norm[0,:]* x_norm[2,:],
                    sqrt(2)*x_norm[1,:]* x_norm[2,:],
                    x_norm,
                    np.ones_like(x_norm[0,:]) ) )
    
    K = D @ D.T #solve ellipsoid
    
    # The objective is now to solve min <q,Kq>, Tr(Q)=1, Q>=0

    c = x_norm.mean(axis=1) # center after normalization
    
    r2 = x_norm.var(axis=1).sum() #varience of this

    #starting guess for ellipsoid
    u = 1/3 * np.hstack( (1, 1, 1,
                          0, 0, 0,
                          -2*c,
                          (c**2).sum()-r2 ) )
    
    
    # And now go to the Douglas-Rachford (Lions-Mercier) iterative algorithm
    gamma = 10; # parameter gamma
    
    M = gamma*K + np.eye(K.shape[0])   
    p = u
    CF = np.zeros(n_iter+1) #cost function
    
    # Iterative solution
    for k in range(n_iter):
        q = project_on_B(p)
        CF[k] = 0.5* q @ K @ q.T #cost function for each iteration
        
        (solution, res, rank, sing) = np.linalg.lstsq(M, 2*q-p, rcond=None) # solve for parameter p update
        p += solution - q
    
    q = project_on_B(q) 
    CF[-1] = 0.5* q @ K @ q.T #saves last cost function
    
    A2 = np.array([[q[0],          q[3]/sqrt(2),  q[4]/sqrt(2)],
                   [q[3]/sqrt(2),  q[1],          q[5]/sqrt(2)],
                   [q[4]/sqrt(2),  q[5]/sqrt(2),  q[2]]])
    
    b2 = q[6:9]
    c2 = q[9]    
    
    # Go back to initial basis. i.e. transform coordinates from centroid origin back to original
    A = P.T @ A2 @ P
    b = -2* A @ x_mean.T + P.T @ b2.T
    c = (A2 @ P @ x_mean.T).T @ (P @ x_mean.T) - b2.T @ P @ x_mean.T + c2

    #calculate ellipsoid surface
    q = np.hstack( (np.diag(A),
                    sqrt(2)*A[1,0], sqrt(2)*A[2,0], sqrt(2)*A[2,1],
                    b, c) )
    
    q = q / np.sum(np.diag(A)) # normalization to stay on the simplex
    
    return A, b, c, q, CF

In [ ]:
def find_ellipsoid_parameters(A, b, c, verbose=False):
    """Given an ellipsoid E={x in R^3, <Ax,x> + <b,x> + c=0}, finds the center, radii and rotation matrix from A, b & c.
    calls: ellipsoid_triple_to_algebraic"""    
    # Determine the center
    # center = solution of A*x + b/2 = 0 -> -A*x = b/2
    center = np.linalg.lstsq(-1*A, 0.5*b)[0]
    if verbose:
        print('Ellipsoid center: {}'.format(center))
        
    # Determine the radii of the axes
    Amat = ellipsoid_triple_to_algebraic(A, b, c) # put ellipsoid parameters into 4x4 algebraic form
    # Transformation to center
    T = np.eye(4) # 4x4 unity matrix
    T[3,0:3] = center
    R = T @ Amat @ T.T # transform to center
    # Determine eigenvalues
    evals, evecs = eig(R[0:3,0:3] / -R[3,3])
    radii = np.power(abs(evals), -0.5) # radii = 1 / sqrt( abs(evals) )
    if verbose:
        print('Ellipsoid radii: {}'.format(radii))
        
    # Determine the rotation matrix
    rotmat = inv(evecs)
    
    return center, radii, rotmat

In [ ]:
def ellipsoid_triple_to_algebraic(A,b,c):
    """Given A,b,c of an ellipsoid E={x in R^3, <Ax,x> + <b,x> + c=0}, returns the "algebraic form", i.e. a 4x4 matrix."""
    Amat = np.zeros((4,4))
    Amat[0:3,0:3] = A
    Amat[0:3,3] = b/2
    Amat[3,0:3] = b/2
    Amat[3,3] = c
    return Amat
    

In [ ]:
def fit_ellipsoid_iter(x, n_iter=5000, plot_CF=True, return_CF=False):
    """
    Fit an ellipsoid using the approach by Weiss et al.
    
    This is just a wrapper to connect the two main functions.

    x = object points
    CF = cost function

    calls fit_ellipsoid_DR_SVD
    """
    # Make the fit (with SVD normalization for faster convergence, see the paper SI)
    A, b, c, q, CF = fit_ellipsoid_DR_SVD(x, n_iter)

    #abc is ellipsoid maths shape i.e. matrix vector and scalar
    #q is ellipsoid equation
    #CF cost function
    
    # Optional: plot cost function
    if plot_CF:
        plt.figure()
        plt.plot(CF[:-1] - CF.min())
        plt.yscale('log')
        plt.xlabel('Iteration number k'); plt.ylabel('Cost function f1(q(k)) - min f1(q)')
        plt.show()
        
    # Calculate ellipsoid parameters from A, b, c
    center, radii, rotmat = find_ellipsoid_parameters(A, b, c)
    # residual of Cost function
    residual = CF[-1]
    
    if return_CF:
        return center, radii, rotmat, residual, CF
    else:
        return center, radii, rotmat, residual

Check ellipsoid fit

In [ ]:
def check_convergence_ellipsoid_fit(CF, n_iter, conv_frac=0.1, cutoff=1e-17):
    """Check if ellipsoid fit converged."""
    # Determine required convergence area
    conv_area_start = np.round(n_iter*(1-conv_frac)).astype(int)
    # Calculate cost function difference and check for cutoff
    CF_diff = CF - CF[-1]
    if np.max(CF_diff[conv_area_start:]) > cutoff:
        return False
    return True
    

Find minimal distance between query point and ellipsoid

In [ ]:
def generate_ellipsoid_even(radii, center, rotation_matrix, n_points=1000, 
                            return_mesh=False, return_angles=False):
    """
    Generate ellipsoid points or mesh from ellipsoid parameters.
    
    Surface generation found in: https://stackoverflow.com/questions/47485235/i-want-to-make-evenly-distributed-sphere-in-vtk-python
    Initial point generation is done with a Fibonacci lattice / golden spiral 
    approach to get points that are (relatively) evenly distributed on a unit 
    sphere.
    Coordinates are then stretched by the radii, rotated by the rotation matrix 
    and shifted by the center.
    If return_mesh=True, a surface mesh is finally generated from the points 
    using delaunay triangulation implemented in pyvista.


    Parameters
    ----------
    radii : numpy.ndarray
        (3,) array of ellipsoid radii / axis lengths.
    center : numpy.ndarray
        Coordinates of ellipsoid center (3,).
    rotation_matrix : numpy.ndarray
        Ellpsoid rotation matrix.
    n_points : int, optional
        Number of points in final structure. The default is 1000.
    return_mesh : bool, optional
        If True, returns a pyvista mesh generated from the points. The default is False.

    Returns
    -------
    points : numpy.ndarray, if return_mesh = False (default)
        A (n_points,3) array of ellipsoid points.
        If return_mesh = True, a pyvista mesh of the ellipsoid surface is returned
        instead. The ellipsoid points are then accessible through mesh.points.

    """
    # Generate coordinates distributed by a Fibonacci pattern on the base ellipsoid.    
    indices = np.arange(0, n_points, dtype=float) + 0.5
    
    phi = np.arccos(1 - 2*indices/n_points)
    theta = np.pi * (1 + 5**0.5) * indices
    theta = theta % (2*np.pi) # put theta into range (0, 2*pi)
    
    x =  np.cos(theta) * np.sin(phi) * radii[0]
    y = np.sin(theta) * np.sin(phi) * radii[1]
    z = np.cos(phi) * radii[2]
    
    # combine points
    points = np.c_[x,y,z]
    # rotate points with rotation matrix
    points = np.dot(points, rotation_matrix)
    # Shift points by center
    points += center
    
    if return_angles and not return_mesh:
        return points, theta, phi
    elif return_angles and return_mesh:
        return points_to_surface(points), theta, phi
    
    if return_mesh:
        return points_to_surface(points)
    
    return points

In [ ]:
def minimum_distance_ellipsoid(query_point, center, axes, rotmat, init_angles=(0,0)):
    """
    Find the closest distance and point on an ellipsoid for a query point.
    
    The closest distance and point are found by minimizing the distance function using scipy.optimize.minimize.
    
    Parameters
    ----------
    query_point : numpy.ndarray
        Coordinates of query point.
    center : numpy.ndarray
        Ellipsoid center.
    axes : numpy.ndarray
        Ellipsoid axes.
    rotmat : numpy.ndarray
        Ellipsoid rotation matrix.
    init_angles : tuple, optional
        Angles describing initial guess for closest ellipsoid point to ellipsoid point. The default is (0,0).

    Returns
    -------
    distance : float
        Distance between query point and the determined closest ellipsoid point.
    closest_ellipsoid_point : numpy.ndarray
        Coordinates of closest ellipsoid point.
    res : scipy.optimize.OptimizeResult
        Complete results of scipy.optimize.minimize. Important attributes are:
            res.x : (theta, phi) angles describing the closest ellipsoid point.
            res.fun = distance (closest distance)
            res.success : Boolean flag indicating whether optimization was successful.
        For a complete description see https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.OptimizeResult.html

    Calls:
    distance_to_ellipsoid
    ellipsoid_point_from_angles
    """
    # Minimize the distance over the pair of angles that define the ellipsoid point
    #finds the minimum distance between the query and ellipsoid surface
    minimizing_function = partial(distance_to_ellipsoid, query_point=query_point, center=center, axes=axes, rotmat=rotmat) 
    res = minimize(minimizing_function, init_angles, bounds=((0, 2*np.pi), (0, np.pi)), tol=1e-6)
    # The distance is the minimized function value
    distance = res.fun    
    # Find the closest ellipsoid point using the angles
    closest_ellipsoid_point = ellipsoid_point_from_angles(*res.x, center, axes, rotmat)
    
    return distance, closest_ellipsoid_point, res    

In [ ]:
def distance_to_ellipsoid(angles, query_point, center, axes, rotmat):
    """
    Calculate the distance of a query point to a point on an ellipsoid.
    
    The ellipsoid point is defined by angles and ellipsoid parameters.
    angles: tuple of (theta, phi) with theta in [0,2pi] and phi in [0,pi].
    """  
    # Unpack angles
    theta, phi = angles
    # Generate ellipsoid point from angles
    x =  np.cos(theta) * np.sin(phi) * axes[0]
    y = np.sin(theta) * np.sin(phi) * axes[1]
    z = np.cos(phi) * axes[2]    
    point = np.array([x,y,z])
    # rotate point with rotation matrix and shift it by the center
    point = np.dot(point, rotmat) + center
    
    # Calculate distance to query point
    distance = np.linalg.norm(query_point - point)
    
    return distance

In [ ]:
def ellipsoid_point_from_angles(theta, phi, center, axes, rotmat):
    """
    Generate points on an ellipsoid from the given angles.
    
    Angle ranges: theta 0->2pi, phi 0->pi
    """
    # Generate point from angles
    x =  np.cos(theta) * np.sin(phi) * axes[0]
    y = np.sin(theta) * np.sin(phi) * axes[1]
    z = np.cos(phi) * axes[2]    
    point = np.array([x,y,z])
    # rotate point with rotation matrix and shift it by the center
    point = np.dot(point, rotmat) + center
    
    return point

Ellipsoid RMSE

In [ ]:
def ellipsoid_fit_rmse(image_name, query_points, axes, center, rotmat, return_full=True):
    """
    Calculate the root mean square error for an ellipsoid fit.
    
    For each of the query points, determine the closest distance and corresponding 
    point on the ellipsoid by minimizing the distance function.
    
    Parameters
    ----------
    query_points : numpy.ndarray
        Points to check against the ellipsoid fit.
    axes : numpy.ndarray
        Ellipsoid axes lengths.
    center : numpy.ndarray
        Ellipsoid center.
    rotmat : numpy.ndarray
        Ellipsoid rotation matrix.
    return_full : bool, optional
        Whether to return all distances and nearest ellipsoid points. The default is True.

    Returns
    -------
    ellipsoid_rmse : float
        Root mean square error of ellipsoid fit.
    dist_pe : numpy.ndarray, optional, if return_full=True
        Distances of all query points to ellipsoid.
    ell_points_nearest : numpy.ndarray, optional, if return_full=True
        Nearest ellipsoid points for all query points.

    Calls: 
    generate_ellipsoid_even
    minimum_distance ellipsoid
    """    
    # Find starting angles for all query points
    ellipsoid_points, ell_theta, ell_phi = generate_ellipsoid_even(axes, center, rotmat, 
                                                                   n_points=2000, return_angles=True)
    ellipsoid_angles = np.c_[ell_theta, ell_phi] # Combine ellipsoid angles into array
    
    # Find the INITIAL closest ellipsoid point for each query point and get the corresponding set of angles
    tree_ell = cKDTree(ellipsoid_points) # Make a KDTree
    _, idx_ell0 = tree_ell.query(query_points) # Find the closest points    
    init_angle_array = ellipsoid_angles[idx_ell0] # Find the corresponding angles
 
    # Prepare output arrays
    dist_pe = np.zeros(query_points.shape[0])
    ell_points_nearest = np.zeros(query_points.shape)
    
    # Run minimization. for each query point find closest point on ellipsoid
    for i, point in enumerate(tqdm(query_points)):
        #if i > 10000: break
        init_angles = tuple( init_angle_array[i] )
        dist_pe[i], ell_points_nearest[i,:], res = minimum_distance_ellipsoid(point, center, 
                                                                              axes, rotmat, 
                                                                              init_angles=init_angles)
   
    ##################################################################################################################
    # #visualising this
    # num_points_to_plot = 50
    # indices = np.random.choice(len(query_points), size=num_points_to_plot, replace=False)

    # query_subset = query_points[indices]
    # closest_subset = ellipsoid_points[idx_ell0[indices]]

    # Create the plotter
    # pv.set_jupyter_backend('static')

    # plotter = pv.Plotter()

    # plotter.add_mesh(ellipsoid_points, color='lightblue', opacity=0.5, show_edges=True)
    
    # # Add query points (as a PyVista point cloud)
    # query_points_pv = pv.PolyData(query_points[indices])
    # plotter.add_points(query_points_pv, color='red', point_size=10, render_points_as_spheres=True, label='Data')

    # # starting point
    # int_closest_points_pv = pv.PolyData(closest_subset)
    # plotter.add_points(int_closest_points_pv, color='yellow', point_size=10, render_points_as_spheres=True, label='Initial Guess of Closest Point')
    
    # # Add closest points on ellipsoid
    # closest_points_pv = pv.PolyData(ell_points_nearest[indices])
    # plotter.add_points(closest_points_pv, color='green', point_size=10, render_points_as_spheres=True, label='Closest Point on Fitted Ellipsoid')

    # # Create lines between corresponding points - query and initial
    # lines = []
    # points_combined = []
    
    # for i, (qp, cp) in enumerate(zip(query_subset, closest_subset)):
    #     # Indices for the start and end of the line segment in points_combined
    #     start_idx = 2 * i
    #     end_idx = start_idx + 1
        
    #     points_combined.append(qp)
    #     points_combined.append(cp)
        
    #     # Line format: [number_of_points_in_line, start_point_idx, end_point_idx]
    #     lines.extend([2, start_idx, end_idx])
    
    # points_combined = np.array(points_combined)
    # lines = np.array(lines)
    
    # # Create a PolyData with points and lines
    # lines_poly = pv.PolyData()
    # lines_poly.points = points_combined
    # lines_poly.lines = lines
    
    # # Add the lines to the plotter
    # plotter.add_mesh(lines_poly, color='light_gray', line_width=1)
    
    # # Create lines between corresponding points, query and final
    # f_lines = []
    # f_points_combined = []
    
    # for i, (qp, cp) in enumerate(zip(query_subset, ell_points_nearest[indices])):
    #     # Indices for the start and end of the line segment in points_combined
    #     start_idx = 2 * i
    #     end_idx = start_idx + 1
        
    #     f_points_combined.append(qp)
    #     f_points_combined.append(cp)
        
    #     # Line format: [number_of_points_in_line, start_point_idx, end_point_idx]
    #     f_lines.extend([2, start_idx, end_idx])
    
    # f_points_combined = np.array(f_points_combined)
    # f_lines = np.array(f_lines)
    
    # # Create a PolyData with points and lines
    # f_lines_poly = pv.PolyData()
    # f_lines_poly.points = f_points_combined
    # f_lines_poly.lines = f_lines
    
    # # Add the lines to the plotter
    # plotter.add_mesh(f_lines_poly, color='black', line_width=1)

    # plotter.add_legend()
    # plotter.show()

    # # Save instead of show
    # output_dir = Path("outputs")
    # output_dir.mkdir(exist_ok=True)
    # plot_path = output_dir / f"{image_name}_ellipsoid_fit_visualisation.png"
    # plotter.screenshot(str(plot_path))
    # plotter.close()
    
    # print(f"Saved plot to {plot_path.resolve()}")
    #####################################################################################################################################

    # Calculate the rmse
    residual_sum_squares = np.sum( np.power(dist_pe, 2) )
    ellipsoid_rmse = np.sqrt(residual_sum_squares / len(dist_pe))
    
    if return_full:
        return ellipsoid_rmse, dist_pe, ell_points_nearest

    return ellipsoid_rmse

Ellipsoid characteristics

In [ ]:
def ellipsoid_SA_Vol(axes):
    # Surface area of ellipsoid
    """
    Approximation of ellipsoid surface area after Knud Thomsen (2004).

    Maximum error is +-1.061%    
    Source: www.numericana.com/answer/ellipsoid.htm 
    cited e.g. in https://doi.org/10.1016/j.mri.2008.07.018
    """   
    a, b, c = axes
    p = 1.6075 # constant defined by K. Thomsen
    area = 4 * np.pi* (( (a*b)**p + (a*c)**p + (b*c)**p ) / 3)**(1/p)  
    
    """Volume of an ellipsoid."""
    V = 4/3 * np.pi * axes[0] * axes[1] * axes[2] # Volume is 4/3*pi*abc
      

    return area, V 

In [ ]:
def sphericity_index_ellipsoid(axes):
    """Sphericity index of an ellipsoid as defined in Cruz-Matias et al. 2019, doi: 10.1016/j.jocs.2018.11.005.""" 
    axis_c, axis_b, axis_a = np.sort(axes) 
    sphericity_index = (axis_c**2 / (axis_a*axis_b))**(1/3)
    
    return sphericity_index

In [ ]:
def sphericity_of_ellipsoid(axes, ellipsoid_surface_area, V_ellipsoid, verbose=False):
    """
    Calculate the 'classical' sphericity of an ellipsoid.
    
    Sphericity is defined as surface area of sphere with same volume 
    divided by actual surface area.
    """
    # Surface area of corresponding sphere
    r_csphere = ( (3*V_ellipsoid ) / (4 * np.pi) )**(1/3) # radius of corresponding sphere
    A_csphere =  4 * np.pi * r_csphere**2 # Area of corresponding sphere, 4 pi r^2
    if verbose:
        print("Surface area of sphere with same volume: {}".format(A_csphere))
    # Sphericity    
    sphericity = A_csphere / ellipsoid_surface_area
    if verbose:
        print("The sphericity of the best-fitting ellipsoid is {}".format(sphericity))
    
    return sphericity

Generate a mesh from a point cloud for plotting

In [ ]:
def generate_sphere_even(radius, center, n_points=1000, return_mesh=False):
    """
    Generate points distributed relatively evenly on a sphere.
    
    The points follow a Fibonacci / golden spiral.
    """
    indices = np.arange(0, n_points, dtype=float) + 0.5
    
    phi = np.arccos(1 - 2*indices/n_points)
    theta = np.pi * (1 + 5**0.5) * indices
    
    x =  np.cos(theta) * np.sin(phi) * radius
    y = np.sin(theta) * np.sin(phi) * radius
    z = np.cos(phi) * radius
    
    # combine points
    points = np.c_[x,y,z]
    # Shift points by center
    points += center
    
    if return_mesh:
        return points_to_surface(points)
    
    return points
    

In [ ]:
def points_to_surface(points):
    """Turn point array into pyvista mesh.
    
    Code found in https://stackoverflow.com/questions/47485235/i-want-to-make-evenly-distributed-sphere-in-vtk-python
    """
    # Make points into Pyvista PolyData
    point_cloud = pv.PolyData(points)
    # Delaunay triangulation and surface extraction
    surf = point_cloud.delaunay_3d().extract_surface()
    
    return surf

## Input

In [ ]:
# For visualization
window_size_0 = [1000,1000]
cpos = {'points': [(613.2361218667041, -345.6736538983374, 804.1761468432726),
                   (318.0, 625.0, 103.0),
                   (-0.18533229149002745, 0.5377558197272821, 0.8224783401892688)],
       'fits': [(134.05519106408673, 728.3388319306499, -1326.1850713469473),
                  (318.0849037026922, 633.5867890633508, 211.84321658237687),
                  (0.8856876772441696, 0.4577205594903096, -0.07777678186776335)]}


In [ ]:
#folders
image_folder = Path('path to segmented vesicles')

#parameters shared by all images
pix_size = 1 #nm

# Segmentation labels

# Parameters 
n_iter_ellipsoid_fit = 10000 # Check plot if fit converged!
n_iter_ell_run2 = 50000 # If first fit didn't converge, try once more with this number of iterations
ellipsoid_fit_samplesize = 2000

clean_points_clustering = False # Whether to find outlier points with clustering

# Results dictionary for comparison
output_dir = Path("where to save outputs")
output_dir.mkdir(exist_ok=True)
npy_file_path = image_folder / 'results.npy'
# Check if the file exists
if npy_file_path.exists():
    print("File exists.")
    data = np.load(npy_file_path, allow_pickle=True).item()

else:
    print("File does not exist.")
    print("Creating results.npy")
    data = {}


    

In [ ]:
%%time 
# time check

# initialise overall dict
d_overall = {}

# Loop through image files and apply load segmentation function
for image_path in image_folder.glob('*.mrc'):  # Adjust extension if needed
    #load image
    image_name = image_path.name
    print(image_name)
    print('')
    name_without_ext = image_name.rsplit('.', 1)[0]
    tag = name_without_ext.rsplit('-', 1)[-1]
    print(tag)
    print('')
    
    points = load_segmentation_tif(image_name, tag, verbose=True)
    
    #Plot
    pv.set_jupyter_backend('static')
    p0 = pv.Plotter(notebook=True, window_size=(1000,1000))
    p0.enable_eye_dome_lighting()
    
    if tag == "COP2":
        colour = 'light_green'
    elif tag == "COP1":
        colour = 'pink'
    else:
        colour = 'blue'
    
    for i, (key, p) in enumerate(points.items()):
        p0.add_mesh(p, color = colour, label=key)
    p0.add_legend()
    # p0.camera_position = cpos['points']
    # _ = p0.show()
    # Save screenshot instead of showing plot
    output_file = output_dir / f"{image_name}_input.png"
    p0.screenshot(str(output_file), window_size=(1000, 1000))
    p0.close()  # Optional: close the plotter to free memory
    
    print(f"Saved plot to: {output_file.resolve()}")
    print('')

    # Sphere and ellipsoid fits
    # Dictionary for results
    d_fit = {}
    d_fit['Image_Name'] = image_name
    d_fit['Image_Tag'] = tag
    
    # Results for the tag of that image
    print(f'Fitting of refined {tag} points.')
    p_tmp = points[tag]
    dt = {} # temporary dict

    # Sphere fit
    dt["sphere_center"], dt["sphere_radius"], dt["sphere_rmse"] = ls_sphere_3(p_tmp) # least squares fit
    dt["sphere_radius_nm"] = dt["sphere_radius"]*pix_size
    print("Sphere fit: {} membrane radius is {} nm".format(tag, dt["sphere_radius_nm"]))
    print("Sphere fit error is {} nm".format(dt["sphere_rmse"]*pix_size))
    print('')
    
    # Ellipsoid fit

    # subsample to make things faster 
    n_points = p_tmp.shape[0]
    if n_points > ellipsoid_fit_samplesize: #defined during inputs
        points_sorted = p_tmp[ np.argsort(p_tmp[:,2]), : ] # sort points by z value
        sample_step = int(n_points // ellipsoid_fit_samplesize) #divide and round down
        points_sample = p_tmp[0::sample_step].copy() # samples points according to the above sample step. 
    else:
        points_sample = p_tmp #if our object is smaller than our sample size just use it all :) 

    
    print('Ellipsoid fitting')
    ell_res = fit_ellipsoid_iter(points_sample, n_iter = n_iter_ellipsoid_fit, plot_CF=True, return_CF=True) 
    #n_iter_ellipsoid_fit is number of iterations to fit elipsoid - defined at input.
    #fit_ellipsoid_iter is a function which plots the outcome of ellipsoid fitting. It calls fit_ellipsoid_DR_SVD to actually do the fitting.
    
    #ell_res is output of fit_ellipsoid_iter which is center, radii, rotation matrix of ellipsoid and the residual cost function.
    #below unpacks this into individual dictionary enteries inside dt
    dt["ellipsoid_center"], dt["ellipsoid_axes"], dt["ellipsoid_rotmat"], dt["ellipsoid_residual"], dt["ellipsoid_CF"] = ell_res

     
    # Check convergence of ellipsoid fit, re-run if necessary
    if check_convergence_ellipsoid_fit(dt["ellipsoid_CF"], n_iter_ellipsoid_fit):
        dt["ellipsoid_fit_converged"] = True
        dt["ellipsoid_fit_n_iter"] = n_iter_ellipsoid_fit
    else :
        print('Ellipsoid fit did not converge in first run, re-try with {} iterations.'.format(n_iter_ell_run2))
        ell_res = fit_ellipsoid_iter(points_sample, n_iter = n_iter_ell_run2, plot_CF=True, return_CF=True)
        dt["ellipsoid_center"], dt["ellipsoid_axes"], dt["ellipsoid_rotmat"], dt["ellipsoid_residual"], dt["ellipsoid_CF"] = ell_res
        
        if check_convergence_ellipsoid_fit(dt["ellipsoid_CF"], n_iter_ellipsoid_fit):
            dt["ellipsoid_fit_converged"] = True
            dt["ellipsoid_fit_n_iter"] = n_iter_ell_run2
        else:
            print("Ellipsoid still not converged in run 2.")
            dt["ellipsoid_fit_converged"] = False
            dt["ellipsoid_fit_n_iter"] = n_iter_ell_run2

    #Ellipsoid fits!, lets calculate other stuff
    
    if dt["ellipsoid_fit_converged"]:
        # If ellipsoid fit converged, also calculate rmse and other parameters  

        #RMSE. Find min distance between our data and ellipsoid (points)
        print('Calculating ellipsoid fit rmse...')
        print('')
        dt["ellipsoid_rmse"] = ellipsoid_fit_rmse(image_name, p_tmp, dt["ellipsoid_axes"], dt["ellipsoid_center"], 
                                                  dt["ellipsoid_rotmat"], return_full=False)    
        print('Ellipsoid fit rmse is {} nm.'.format(dt["ellipsoid_rmse"]*pix_size))
        dt["ellipsoid_axes_nm"] = dt["ellipsoid_axes"]*pix_size

        dt["Surface_Area"], dt["Ellipsoid_Volume"] = ellipsoid_SA_Vol(dt["ellipsoid_axes"])

        dt["Surface_Area_nm"] = dt["Surface_Area"]*pix_size
        dt["Ellipsoid_Volume_nm"] = dt["Ellipsoid_Volume"]*pix_size

        print("Ellipsoid fit: {} membrane axes is {} nm".format(tag, dt["ellipsoid_axes_nm"]))
        print("Ellipsoid fit: {} membrane Surface Area is {} nm".format(tag, dt["Surface_Area_nm"]))
        print("Ellipsoid fit: {} membrane Volume is {} nm".format(tag, dt["Ellipsoid_Volume_nm"]))

        # Sphericity (two measures, sphericity index and "classical" sphericity)                
        dt["sphericity_index"] = sphericity_index_ellipsoid(dt["ellipsoid_axes"])
        print("Sphericity index of best-fitting ellipsoid is {} for {} membrane.".format(dt["sphericity_index"], tag))
        
        dt["sphericity_classical"] = sphericity_of_ellipsoid(dt["ellipsoid_axes"], dt["Surface_Area"], dt["Ellipsoid_Volume"], verbose=False)
        print("Sphericity of best-fitting ellipsoid is {} for {} membrane.".format(dt["sphericity_classical"], tag))


    # Save results into dictionary
    d_fit['info'] = dt

    # Plot
    surf_ell = generate_ellipsoid_even(d_fit['info']['ellipsoid_axes'], 
                                       d_fit['info']['ellipsoid_center'], 
                                       d_fit['info']['ellipsoid_rotmat'], 
                                       n_points=1000, 
                                       return_mesh=True, return_angles=False)
    
    surf_sphere = generate_sphere_even(d_fit['info']['sphere_radius'], 
                                       d_fit['info']['sphere_center'], 
                                       n_points=1000, 
                                       return_mesh=True)

    if dt["ellipsoid_fit_converged"]:

        p0 = pv.Plotter(notebook=True, window_size=(1000,1000))
        p0.enable_eye_dome_lighting()
        p0.enable_depth_peeling()
        p0.add_mesh(points[tag], color=colour, label=tag, render_points_as_spheres=True, point_size=50)
        p0.add_mesh(surf_sphere, color='lightyellow', label='sphere fit')
        p0.add_mesh(surf_ell, style='wireframe', color='black', label='ellipsoid fit')
        p0.add_legend()
        
        # p0.camera_position = cpos['fits']
        # _ = p0.show()
        plot_path = output_dir / f"{image_name}_result.png"
        p0.screenshot(str(plot_path))
        p0.close()
        
    else:
        p0 = pv.Plotter(notebook=True, window_size=(1000,1000))
        p0.enable_eye_dome_lighting()
        p0.enable_depth_peeling()
        p0.add_mesh(points[tag], color=colour, label=tag, render_points_as_spheres=True, point_size=50)
        p0.add_mesh(surf_sphere, color='lightyellow', label='sphere fit')
        p0.add_legend()
        
        # p0.camera_position = cpos['fits']
        # _ = p0.show()
        plot_path = output_dir / f"{image_name}_result.png"
        p0.screenshot(str(plot_path))
        p0.close()
    
    # print out the dict
    print('')
    print(d_fit)
    print('') # added this to break a line between print-outs
    
    #save to overall dict
    d_overall[image_name] = d_fit

    np.save('results.npy', d_overall)

## Save results or compare to previously saved results